# Hate Speech Detector - EN - Features extraction for Conv. & LSTM model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'lstm'

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/Reddit/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/Reddit/clean_data.csv', 'r') as f:
        for d in list(csv.reader(f))[1:]:
            tweets.append(d[1])  # tweet
            labels.append(d[2])  # class
    with open('hsd/Reddit/perfect_data.pkl', 'wb') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/Reddit/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 79915
Labels: 79915


In [5]:
list(zip(tweets[:5], labels[:5]))

[("i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.",
  0),
 ('This is what the left is really scared of.    https://redd.it/9rfkts', 0),
 ('That literally looks like a monkey. Why are we supposed to pretend it’s a person bc it’s wearing a red hat?',
  0),
 ('Dumb Cunt', 0),
 ('It makes you an asshole.', 0)]

## Features extraction

In [6]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    # stop_words = set(stopwords.words('english'))
    # words = [w for w in words if not w in stop_words]
    tags = nltk.pos_tag(words)
    # words = [lemmatizer.lemmatize(w[0]) if pos_tagger(w[1]) is None else lemmatizer.lemmatize(w[0], pos_tagger(w[1])) for w in tags]
    tags = [x[1] for x in tags]
    return words, tags

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = text_string.encode('ascii', 'ignore').decode('ascii')
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = re.sub(hashtag_regex, '', parsed_text)
    list_words, tag_list = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    tag_str = ' '.join(tag_list)
    return parsed_text, tag_str

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

# def get_pos_string(tweet):
#     text = preprocess(tweet)
#     tokens = word_tokenize(text)
#     tags = nltk.pos_tag(tokens)
#     tag_list = [x[1] for x in tags]
#     tag_str = ' '.join(tag_list)
    
    # return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

### Median sentences length

In [7]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in data:
        sentence, pos_string = preprocess(d)
        # sentence = preprocess(d)
        # pos_string = get_pos_string(d)
        all_lengths.append(len(sentence.split(' ')))
        all_lengths.append(len(pos_string.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        pos_lengths.append(len(pos_string.split(' ')))
    
    return int(np.median(all_lengths)), int(np.median(wt_lengths)), int(np.median(pos_lengths))

In [8]:
opt_length, opt_wt_length, opt_pos_length = median_sentences_length(tweets)
dim = 6*50 if MODEL == 'conv' else 300

print('Optimal all length: {}'.format(opt_length))
print('Optimal sentence length: {}'.format(opt_wt_length))
print('Optimal pos sentence length: {}'.format(opt_pos_length))

Optimal all length: 16
Optimal sentence length: 16
Optimal pos sentence length: 16


### Supervised fastText wordtokens training

In [9]:
if not os.path.exists('hsd/Reddit/fasttext.ft'):
    with open('hsd/Reddit/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)[0]))

# load fasttext model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/Reddit/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [10]:
def get_wordtoken_fts(data, length):
    
    sentences_words = []
    for d in data:
        sentence = preprocess(d)[0]
        sentences_words.append(sentence.split(' '))
    
    sentences_words = [pad_words(sw, length) for sw in sentences_words]
    vecs = Magnitude(MagnitudeUtils.download_model('http://magnitude.plasticity.ai/fasttext/medium/crawl-300d-2M.magnitude'))
    
    ft_matrices = []
    for sw in sentences_words:
        ft_matrix = []
        for w in sw:
            ft_matrix.append(vecs.query(w))
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [11]:
wordtoken_features = get_wordtoken_fts(tweets, opt_wt_length)

In [12]:
wordtoken_features[0]

02, -1.139940e-02,  1.323700e-02, -5.497250e-02,
        -1.560410e-02, -5.590690e-02, -2.292340e-02,  1.317470e-02,
        -7.879910e-02,  4.790240e-02,  1.438940e-02,  4.154860e-02,
         2.152180e-02,  1.611488e-01, -2.192670e-02,  4.126830e-02,
         2.828050e-02, -4.276330e-02, -8.565100e-03, -2.700350e-02,
         3.450960e-02,  1.414646e-01,  1.554180e-02, -7.721070e-02,
         3.189962e-01, -4.313710e-02,  5.740190e-02,  1.837600e-03,
        -1.174200e-02,  1.694340e-02, -1.342390e-02, -1.753510e-02,
         5.917700e-03, -2.289220e-02,  1.127480e-02, -8.073010e-02,
        -4.609590e-02,  4.889900e-02,  9.764240e-02, -1.488770e-02,
         2.223820e-02, -5.858540e-02,  1.549819e-01, -1.055850e-02,
         4.329300e-03,  1.102564e-01, -5.568880e-02, -9.991600e-02,
        -2.238766e-01, -1.148037e-01,  1.155512e-01,  6.032960e-02,
         1.984927e-01, -2.865420e-02,  7.534190e-02,  2.416920e-02,
        -3.074100e-02, -6.836520e-02, -3.485220e-02,  9.250300e-03,

### Supervised fastText pos training

In [13]:
if not os.path.exists('hsd/Reddit/fasttext_pos.ft'):
    with open('hsd/Reddit/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)[1]))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/Reddit/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [14]:
def get_pos_fts(data, length):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in data:
        pos_string = preprocess(d)[1]
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    pos_tags = [pad_words(pt, length) for pt in pos_tags]
    vecs = Magnitude(MagnitudeUtils.download_model('http://magnitude.plasticity.ai/fasttext/medium/crawl-300d-2M.magnitude'))
    
    ft_matrices = []
    for pt in pos_tags:
        ft_matrix = []
        for t in pt:
            ft_matrix.append(vecs.query(t))
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [15]:
# pos_features = get_pos_fts(tweets, opt_pos_length)

In [16]:
# pos_features[0]

In [17]:
np.array(wordtoken_features).shape

(79915, 16, 300)

In [18]:
# np.array(pos_features).shape

### All features

In [19]:
# #Now join them all up
features = np.array(wordtoken_features)
# features = np.concatenate([wordtoken_features, pos_features], axis=1)

In [20]:
features.shape

(79915, 16, 300)

In [21]:
features[0]


array([[-0.13831291,  0.0253904 , -0.0224353 , ..., -0.0090946 ,
        -0.0456453 , -0.0373253 ],
       [ 0.0305197 , -0.0764269 , -0.0918143 , ...,  0.052006  ,
         0.0136522 , -0.043585  ],
       [-0.0265963 , -0.0167253 , -0.0688487 , ..., -0.0375365 ,
        -0.0471593 , -0.0073507 ],
       ...,
       [-0.0374463 , -0.040049  ,  0.0037414 , ..., -0.1019281 ,
         0.0126231 ,  0.0015616 ],
       [-0.0579957 ,  0.0199212 ,  0.0074381 , ...,  0.0494365 ,
         0.088438  , -0.0123968 ],
       [-0.0022412 , -0.1313298 ,  0.0091819 , ..., -0.1006392 ,
        -0.0397279 ,  0.0054947 ]])

## Save features & labels

In [22]:
archive = dir_archive('hsd/Reddit/X_y_{}'.format(MODEL), {'features': features, 'labels': labels,
                                                                'wt_num': np.array(wordtoken_features).shape[1]}, serialized=True)
archive.dump()
del archive